In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import seaborn
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from pycaret.classification import *
from pycaret.classification import evaluate_model
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import selenium
from selenium import webdriver
import random
from selenium.webdriver.common.by import By
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
pd.set_option('display.max_columns', 100)

In [3]:
def kelly(p, b, weight):
    stake = round((p * (b - 1) - (1 - p)) / (b - 1), 4) * weight
    return stake if stake > 0.0 else 'no value'

In [50]:
today = f"{datetime.now():%Y%m%d}"
setup_df = pd.read_csv(f'data_20220911.csv', index_col = [0])
setup_df = setup_df[setup_df['odds_checker_sum'] > 1.03]
print(setup_df.shape)

(322, 51)


In [38]:
setup_df.columns

Index(['Expected goals (xG)', 'Total shots', 'Big chances',
       'Big chances missed', 'Accurate passes', 'Accurate passes percentage',
       'Fouls committed', 'Offsides', 'Corners', 'Shots off target',
       'Shots on target', 'Blocked shots', 'Hit woodwork', 'Shots inside box',
       'Shots outside box', 'xG open play', 'xG set play',
       'xG on target (xGOT)', 'Passes', 'Own half', 'Opposition half',
       'Accurate long balls', 'Accurate long balls percentage',
       'Accurate crosses', 'Accurate crosses percentage', 'Throws',
       'Yellow cards', 'Red cards', 'Tackles won', 'Tackles won percentage',
       'Interceptions', 'Blocks', 'Clearances', 'Keeper saves', 'Duels won',
       'Ground duels won', 'Ground duels won percentage', 'Aerial duels won',
       'Aerial duels won percentage', 'Successful dribbles',
       'Successful dribbles percentage', 'score', 'posession', 'full_odds',
       'xG penalty', 'odds_checker_sum', 'odds_predict', '0', '1', '2',
       'tar

In [51]:
setup_df = setup_df[['Expected goals (xG)','Shots on target', 'Total shots', 'Big chances','Accurate passes percentage',
        'xG on target (xGOT)', 'Passes', 'Own half', 'Opposition half','Red cards','score', 'posession', 'odds_checker_sum',
        '0', '1', '2','target']]

In [35]:
setup_df

,Expected goals (xG),Total shots,Big chances,Accurate passes percentage,xG on target (xGOT),Passes,Own half,Opposition half,Red cards,score,posession,0,1,2,target
0,0.89,7.0,1.0,0.06,0.28,77.0,43.0,34.0,0.0,0.0,20.0,2.300000,2.387500,4.200000,0
1,-1.21,-5.0,-1.0,-0.09,-1.28,-49.0,5.0,-59.0,0.0,-1.0,-16.0,4.750000,13.000000,1.250000,2
2,-0.74,-3.0,-1.0,-0.11,-1.25,-141.0,-31.0,-109.0,0.0,-2.0,-26.0,20.000000,56.000000,1.024390,2
3,0.37,4.0,1.0,0.04,0.97,56.0,-14.0,70.0,0.0,-1.0,12.0,3.500000,6.500000,1.600000,1
5,-0.95,1.0,-2.0,-0.02,-1.99,-19.0,-59.0,38.0,0.0,-1.0,-4.0,3.126923,4.500000,1.816667,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,-0.63,3.0,-5.0,0.01,-0.55,-1.0,-8.0,9.0,0.0,0.0,0.0,2.500000,3.300000,2.647059,1
339,0.65,8.0,1.0,0.07,0.08,75.0,33.0,41.0,0.0,0.0,14.0,3.112500,1.615385,6.750000,1
340,-1.22,-9.0,-2.0,-0.03,-1.06,-20.0,-31.0,7.0,0.0,-2.0,-4.0,19.000000,61.000000,1.025000,2
341,0.21,6.0,0.0,0.09,0.19,6.0,20.0,4.0,0.0,1.0,2.0,3.800000,1.368421,9.500000,2


In [68]:
grid = setup(data = setup_df, target='target', session_id = 123,
            polynomial_features = True, feature_interaction = True, 
            feature_ratio = True, trigonometry_features = True, create_clusters = True)

,Description,Value
0,session_id,123
1,Target,target
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(322, 17)"
5,Missing Values,True
6,Numeric Features,16
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [69]:
best = compare_models()
tuned = tune_model(best)
print(tuned)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7391,0.8752,0.7492,0.7971,0.7494,0.6124,0.6283
1,0.7826,0.8491,0.7381,0.8551,0.7754,0.6472,0.6923
2,0.6522,0.8182,0.6175,0.6746,0.6335,0.4651,0.4838
3,0.5652,0.7174,0.5429,0.5732,0.5591,0.3114,0.3185
4,0.5217,0.7072,0.4937,0.5217,0.5217,0.2645,0.2645
5,0.7727,0.8782,0.7672,0.7727,0.7727,0.6541,0.6541
6,0.7273,0.8458,0.6714,0.7197,0.7128,0.5600,0.5682
7,0.4091,0.5847,0.3429,0.3727,0.3773,0.0104,0.0112
8,0.6818,0.8249,0.6429,0.7079,0.6768,0.5048,0.5165


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.3, loss='deviance', max_depth=5,
                           max_features=1.0, max_leaf_nodes=None,
                           min_impurity_decrease=0.0005,
                           min_impurity_split=None, min_samples_leaf=3,
                           min_samples_split=7, min_weight_fraction_leaf=0.0,
                           n_estimators=160, n_iter_no_change=None,
                           presort='deprecated', random_state=123,
                           subsample=0.45, tol=0.0001, validation_fraction=0.1,
                           verbose=0, warm_start=False)


In [59]:
xgb = create_model('xgboost')
tuned = tune_model(xgb)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6957,0.7879,0.6714,0.7003,0.6958,0.5279,0.5294
1,0.6957,0.7802,0.6571,0.7652,0.6899,0.5061,0.5414
2,0.5652,0.7593,0.5063,0.5401,0.5379,0.3134,0.3226
3,0.5217,0.6087,0.5095,0.5435,0.5241,0.2470,0.2502
4,0.6522,0.8090,0.6381,0.7066,0.6540,0.4556,0.4656
5,0.6364,0.7743,0.6349,0.6826,0.6463,0.4551,0.4652
6,0.5455,0.7575,0.5048,0.6209,0.5709,0.3189,0.3260
7,0.3182,0.3519,0.2619,0.3214,0.3051,-0.1149,-0.1207
8,0.6364,0.8598,0.6048,0.6433,0.6380,0.4377,0.4391


In [63]:
lgbm = create_model('lightgbm')
tuned = tune_model(lgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6087,0.7994,0.5603,0.5719,0.5778,0.3765,0.3878
1,0.5652,0.8387,0.4905,0.4331,0.4771,0.2744,0.3284
2,0.6522,0.8160,0.5794,0.6457,0.6026,0.4286,0.4763
3,0.5217,0.6057,0.4651,0.6304,0.4826,0.2118,0.2387
4,0.6087,0.7621,0.5603,0.5894,0.5716,0.3802,0.3953
5,0.6364,0.7954,0.5979,0.6427,0.6187,0.4286,0.4472
6,0.6364,0.7590,0.5524,0.5949,0.6074,0.4304,0.4406
7,0.4545,0.4630,0.3619,0.4260,0.3966,0.0671,0.0800
8,0.7273,0.8481,0.7048,0.7273,0.7273,0.5742,0.5742


In [70]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [71]:
predicted_df = predict_model(tuned, raw_score = True)
z = predicted_df
predicted_df = predicted_df[['0','1','2'] + list(predicted_df.columns[-5:])]
bo_po_dha = list(zip(predicted_df.columns[0:3],predicted_df.columns[-3:],['draw','home','away']))
for book_odds, pred_odds, dha in bo_po_dha:
    predicted_df[dha] = predicted_df.apply(lambda x: kelly(x[pred_odds], x[book_odds], 0.25), axis=1)
predicted_df.to_csv('pred_profits.csv')
df = predicted_df
dha = {'draw':'0','home':'1','away':'2'}
df = df.replace('no value', 0)
df[['draw','home','away']] = df[['draw','home','away']].astype(float)
for word, number in dha.items():
    df[f'{word}_profits'] = np.where(df['target'] == int(number),
                                    df[word] * (df[number] - 1) * 100,
                                    -100*df[word])
df['profits'] = df[df.columns[-3:]].sum(axis = 1)
p = round(np.sum(df["profits"]), 3)
df = df.round(3)
df.to_csv(f'profits_{datetime.now():%Y%m%d}.csv')
print(f'number of bets made: {df.shape[0]}')
print(f'profit on $100 bankroll: ${p}')
print(df.sum()[-4:-1])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.5979,0.7994,0.5370,0.5880,0.5928,0.3741,0.3743


number of bets made: 97
profit on $100 bankroll: $41.603
draw_profits   -130.128
home_profits      8.008
away_profits    163.718
dtype: float64


In [66]:
df

,0,1,2,target,Label,Score_0,Score_1,Score_2,draw,home,away,draw_profits,home_profits,away_profits,profits
0,3.900,4.333,1.683,2,1,0.227,0.419,0.354,0.000,0.061,0.000,-0.000,-6.118,0.000,-6.118
1,13.000,1.050,41.000,1,1,0.113,0.728,0.159,0.010,0.000,0.034,-0.972,0.000,-3.448,-4.420
2,5.200,1.222,13.000,1,1,0.142,0.654,0.204,0.000,0.000,0.035,-0.000,0.000,-3.452,-3.452
3,3.500,1.533,7.375,2,1,0.174,0.620,0.207,0.000,0.000,0.021,-0.000,-0.000,13.085,13.085
4,9.000,1.111,21.000,1,1,0.136,0.692,0.172,0.007,0.000,0.033,-0.708,0.000,-3.270,-3.978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,3.112,1.615,6.750,1,1,0.224,0.538,0.237,0.000,0.000,0.026,-0.000,0.000,-2.620,-2.620
93,4.200,1.400,9.625,1,1,0.115,0.729,0.156,0.000,0.013,0.015,-0.000,0.518,-1.460,-0.942
94,4.600,12.500,1.286,2,2,0.253,0.210,0.536,0.011,0.035,0.000,-1.138,-3.545,0.000,-4.682
95,2.700,1.900,5.500,2,1,0.280,0.494,0.226,0.000,0.000,0.014,-0.000,-0.000,6.131,6.131
